# Explore IBL data Pipeline

This notebook gives an brief introduction on the IBL pipeline, and provide some guidance on how to explore the data pipeline with DataJoint tools

*DataJoint* is a general framework in Python (or Matlab) that allow users to interact with MySQL database with Python (or Matlab). To learn more, visit the website https://datajoint.io/, DataJoint Neuro is the team that develop DataJoint and help researchers to adopt DataJoint and design pipeline. To check out more, please visit https://djneuro.io/

Using DataJoint framework, we have been processing data daily within IBL project, and provides a website that display the daily training results at https://data.internationalbrainlab.org

Here we will go through some useful tools to navigate with the data itself.  

First thing first, let's **import DataJoint and the IBL pipeline package**.

In [1]:
import datajoint as dj

In [ ]:
from ibl_pipeline import reference, subject, action, acquisition, data, behavior
from ibl_pipeline.analyses import behavior as behavior_analyses

# 1. Browse schemas you have access to - `dj.list_schemas()`

Schemas are groups of tables. Tables in one schema are intuitively related.

The IBL database current have the following schemas, which could be listed with a DataJoint method:

In [ ]:
dj.list_schemas()

## Major schemas:   

1. Meta data from **Alyx**:   
    1) `ibl_reference`: general information on labs, users and projects  
    2) `ibl_subject`: subject related housing, genetic, information  
    3) `ibl_action`: surgery information  
    4) `ibl_acquisition`: session information  
    5) `ibl_data`: data managing information  
    
2. Imported data from **FlatIron**: `ibl_behavior`, trial information of the task.
3. Computed analyzed results of the behavioral data: `ibl_analyses_behavior`    

# 2. Browse tables in a schema - `dj.Diagram` or `dj.Di`  

`dj.Diagram` shows the tables and the dependencies among them. There are four types of DataJoint tables:

**Table tiers**:  

_Manual table_: green box, entries in a manual table were inserted manually and cannot recovered if deleted. A typical example of a manual table is the subject table that saves information of animal subjects  

**Lookup table**: gray box, lookup tables save general facts that are rarely changed, such as possible genetic lines of an animals, or parameter information of a model  

**Imported table**: blue oval, entries in an imported table are imported from the external data, usually with code within the DataJoint table. If deleted, the entries are recoverable if external data still exists. Examples of imported tables are tables for experimental recordings.  

**Computed table**: red circle, entries in an imported table are computed from data in the database. If deleted, the entries are easily recoverable. Examples of computed tables are tables for analyzed results.

**Part table**: plain text, tightly link to its master table, usually imported through the master table.

**Dependencies**:  
**One-to-one primary**: thick solid line, tables with one-to-one primary dependency have the exact same definition of the primary key.  

**One-to-many primary**: thin solid line, the child table inherits the primary key definition from its parent, but has additional field as part of the primary key as well.

**Secondary foreign key reference**: dashed line, a secondary attribute references to a 
Renamed secondary foreign key references: orange dot

## Show tables in the whole schema

In [ ]:
import datajoint as dj
from ibl_pipeline import reference

In [ ]:
dj.Diagram(reference)

## Show diagram of arbitruary parts of the database

A combination of arbitruary tables:

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(subject.Line) + dj.Diagram(subject.SubjectLab) + dj.Diagram(subject.SubjectProject)

The tables could be from different schemas:

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(acquisition.Session)

# 3. Getting the detailed definition of a table - `table.describe()`

In [ ]:
subject.Subject.describe();

# 4. Browsing of data - queries

Query all subjects

In [ ]:
subject.Subject()

## Restriction  `&`: filtering data

### Restriction: Query one subject

In [ ]:
# restrict by string
subject.Subject & 'subject_nickname="IBL-T1"'

In [ ]:
# restrict by dictionary
from uuid import UUID
subject.Subject & {'subject_uuid': UUID('00c60db3-74c3-4ee2-9df9-2c84acf84e92')}

In [ ]:
subject.Subject & {'sex': 'm'}

### Restriction: Query subjects born after a date

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"'

### Restriction: subjects within a range of dates

In [ ]:
subject.Subject & 'subject_birth_date between "2019-01-01" and "2019-04-01"'

### Restriction: Query subjects on multiple attributes

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"' & 'sex="M"'

### Restriction: Query dead subjects

In [ ]:
# subjects that are dead
subject.Subject & subject.Death

In [ ]:
# subjects that are alive
subject.Subject - subject.Death

## Join `*`:  gather information from different tables

In [ ]:
subject.Subject * acquisition.Session

## Projection `.proj()`: focus on attributes of interest

In [ ]:
subject.Subject.proj()

In [ ]:
subject.Subject.proj('subject_birth_date', 'sex')

### rename attribute with ***proj()***

In [ ]:
subject.Subject.proj('sex', dob='subject_birth_date')

### perform simple computations with ***proj***

**Example 1: Get date of a session:**

In [ ]:
sessions_with_date = acquisition.Session.proj(session_date='date(session_start_time)')

In [ ]:
sessions_with_date

**Example 2: Age of the animal when performing each session?**

In [ ]:
# First get the date of birth and the session date into the same query
q = subject.Subject * acquisition.Session

In [ ]:
q

In [ ]:
# Then compute the age
q_with_age = q.proj(age='datediff(session_start_time, subject_birth_date)')
q_with_age

## Aggregation `.aggr()`: simple computation of one table against another table

Example: how many sessions does each subject do so far?

In [ ]:
subject.Subject.aggr(acquisition.Session, 'subject_nickname', n='count(*)')

# 5. Fetching data

## Fetch all fields: `fetch()`

In [ ]:
# fetch all data from a table
subjs = subject.Subject.fetch()
subjs

In [ ]:
subjs['subject_uuid']

In [ ]:
subjs['subject_birth_date']

In [ ]:
# fetch as a list of dictionaries
subjs_dict = subject.Subject.fetch(as_dict=True)
subjs_dict

In [ ]:
# fetch as pandas dataframe
subjs_df = subject.Subject.fetch(format='frame')
subjs_df

In [ ]:
# fetch the primary key
pk = subject.Subject.fetch('KEY')
pk

In [ ]:
# fetch specific attributes
dob, sex = subject.Subject.fetch('subject_birth_date', 'sex')

In [ ]:
dob

In [ ]:
info = subject.Subject.fetch('subject_birth_date', 'sex', as_dict=True)
info

## fetch data only from one entry: `fetch1`

In [ ]:
CSHL_015 = (subject.Subject & {'subject_nickname': 'CSHL_015'}).fetch1('KEY')  # "fetch1()" because we know there's only one

In [ ]:
CSHL_015

In [ ]:
IBL_T1 = (subject.Subject & {'subject_nickname': 'IBL-T1'}).fetch1()

In [ ]:
IBL_T1

# 5. Behavioral trial information

The most important information in the data pipeline so far is the behavioral trials for the 2 alternative force choice task. We would go into a bit more details in the relevant tables.

The `behavior` schema contains tables for the behavior information. The most important table is `behavior.TrialSet`

In [ ]:
behavior.TrialSet()

The TrialSet table stores some summary statistics in one session of behavioral experiment. To also show the information of the subject, we could join the table with subject related tables.

In [ ]:
behavior.TrialSet * subject.Subject * subject.SubjectLab * subject.SubjectProject

We could restrict to one session by:

In [ ]:
import datetime
q = behavior.TrialSet * subject.Subject * subject.SubjectLab * subject.SubjectProject & \
    'subject_nickname="CSHL_015"' & {'session_start_time': datetime.datetime(2019, 9, 16, 13, 44, 46)}
q

The trial-by-trial information are shown in a **part table** `behavior.TrialSet.Trial`

In [ ]:
behavior.TrialSet.Trial()

We could check the documentation of each of the column with `describe()`

In [ ]:
behavior.TrialSet.Trial.describe();

To fetch some part of data, we could do use the fetch method:  

For example we would like to see the stimulus contrasts, and the animal choices in one session (that we already queried and saved the results as q), we could do:

In [ ]:
# we could fetch the fields as a list of dictionary, only for the trials with a choice
data = (behavior.TrialSet.Trial & q & 'trial_response_choice !="No Go"').fetch(
    'trial_stim_contrast_left', 'trial_stim_contrast_right', 'trial_response_choice', as_dict=True)

In [ ]:
# We could then convert the data to a dataframe for further analyses
import pandas as pd
df = pd.DataFrame(data)
df

Then we can do some computation here to explore the data.

We first compute the signed contrast, so that the contrasts on the right are positive

In [ ]:
df['signed_contrast'] = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']

In [ ]:
df['report_right'] = df['trial_response_choice'] == "CCW"

In [ ]:
report_right = df.groupby(['signed_contrast'], as_index=False).mean()
report_right

Then let's plot the psychometric curve: prob_report_right vs signed_contrast:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
plt.plot(report_right['signed_contrast'], report_right['report_right'], 'o')
plt.xlabel('Signed Contrast')
plt.ylabel('Probability reporting right')

Some basic analyses such as psychometric curves have been pre-processed and saved in the downstream tables, such as `behavior_analyses.PsychResults`. In the next notebook, we would like to step through how to fetch data from that table and recreate the psychometric curves for a particular session.